### ASX Dividend Harvesting Analysis - Citi Instalment MINIs

### Import Modules

In [1]:
import yfinance as yf
import plotly.express as px
import pyasx.data.securities
import pyasx.data.companies
from dateutil.relativedelta import relativedelta
import pandas as pd

### ASX Company

In [2]:
# Yahoo Finance only supports ASX company dividend hisotry (exluding ETFs)
asx_code = "BHP"
dps = 2.5

### CitiWarrant Characteristics

In [3]:
annual_interest_rate_on_loan = 0.068
loan_amount_per_share = 20

Warrant Calculation

In [4]:
daily_interst_rate = annual_interest_rate_on_loan / 365
underlying_share_price = pyasx.data.securities.get_security_info(asx_code)["last_price"]
first_instalment = round(underlying_share_price - loan_amount_per_share, 2)
gearing = loan_amount_per_share / underlying_share_price

### Dividend Harvesting Plan

In [5]:
investment_amount = 10000
total_warrants_bought = int(investment_amount / first_instalment)

### Dividend Harvesting Analysis

In [6]:
# number of past dividends/distributions for historical average calculation 
dividend_history = 1000
ex_dates = yf.Ticker(ticker = asx_code + ".AX").dividends[-dividend_history:].index

print(yf.Ticker(ticker = asx_code + ".AX").dividends[-dividend_history:])

if len(ex_dates) > 0:
    average_div_per_share = sum(yf.Ticker(ticker = asx_code + ".AX").dividends[-dividend_history:]) / len(ex_dates)
else:
    div_hisotry_df = pd.read_csv(asx_code + "_Div.csv")
    div_hisotry_df["Date"] = pd.to_datetime(div_hisotry_df["Date"], format = "%d/%m/%Y")
    div_hisotry_df = div_hisotry_df.sort_values("Date")
    ex_dates = list(div_hisotry_df["Date"])
    average_div_per_share = sum(div_hisotry_df["Dividend"]) / len(ex_dates)

# dates before the ex dates
dates_prior_to_ex_dates = []
for date in ex_dates:
    date_prior = date - relativedelta(days= 1)
    dates_prior_to_ex_dates.append(date_prior)

# get the close prices on the dates prior to the ex dates
close_on_dates_prior_to_ex_dates = []
historical_close = yf.download(tickers = asx_code + ".AX")[["Close"]]
for date in dates_prior_to_ex_dates:
    i = len(historical_close) - 1
    while i >= 0:
        if historical_close.index[i] <= date:
            close_on_dates_prior_to_ex_dates.append(historical_close.iloc[i]["Close"])
            break
        i -= 1

# get the date on which the previous dividend amount has been recovered but before the next ex date
div_recover_dates = []
k = 0 #index for close_on_dates_prior_to_ex_dates
for i in range(len(dates_prior_to_ex_dates)):
    if i < len(dates_prior_to_ex_dates) - 1:
        historical_range = pd.DataFrame(historical_close.loc[(historical_close.index > dates_prior_to_ex_dates[i]) & (historical_close.index <= dates_prior_to_ex_dates[i + 1]), :])
        for j in range(len(historical_range)):
            if historical_range.iloc[j]["Close"] >= close_on_dates_prior_to_ex_dates[k]:
                div_recover_dates.append(historical_range.index[j])
                break
            if j == len(historical_range) - 1:
                div_recover_dates.append(None)
        k += 1
    else:
        historical_range = pd.DataFrame(historical_close.loc[(historical_close.index > dates_prior_to_ex_dates[i]), :])
        for j in range(len(historical_range)):
            if historical_range.iloc[j]["Close"] >= close_on_dates_prior_to_ex_dates[k]:
                div_recover_dates.append(historical_range.index[j])
                break
            if j == len(historical_range) - 1:
                div_recover_dates.append(None)
        k += 1

# number of day for dividend to be recovered by the next ex date
div_recovery_duration = []
for i in range(len(div_recover_dates)):
    if div_recover_dates[i] != None:
        days = (div_recover_dates[i] - dates_prior_to_ex_dates[i]).days
        div_recovery_duration.append(days)


# get the date on which the previous dividend amount has been fully recovered (regardless of the next ex date)
full_div_recover_dates = []
k = 0 #index for close_on_dates_prior_to_ex_dates
for i in range(len(dates_prior_to_ex_dates)):
    if i < len(dates_prior_to_ex_dates) - 1:
        historical_range = pd.DataFrame(historical_close.loc[(historical_close.index > dates_prior_to_ex_dates[i]), :])
        for j in range(len(historical_range)):
            if historical_range.iloc[j]["Close"] >= close_on_dates_prior_to_ex_dates[k]:
                full_div_recover_dates.append(historical_range.index[j])
                break
        k += 1

# number of day for dividend to be fully recovered (regardless of the next ex date)
full_div_recovery_duration = []
for i in range(len(full_div_recover_dates)):
    days = (full_div_recover_dates[i] - dates_prior_to_ex_dates[i]).days
    full_div_recovery_duration.append(days)

# recovery stats
full_avg_recovery_duration = sum(full_div_recovery_duration) / len(full_div_recovery_duration)
avg_recovery_duration = sum(div_recovery_duration) / len(div_recovery_duration)
successful_recovery = len(div_recovery_duration)
success_rate = round(successful_recovery / len(ex_dates), 4)

# Investment insights
total_div_received = dps * total_warrants_bought
avg_interest_on_loan = (loan_amount_per_share * (1 + daily_interst_rate) ** full_avg_recovery_duration - loan_amount_per_share) * total_warrants_bought
max_interest_on_loan = (loan_amount_per_share * (1 + daily_interst_rate) ** max(full_div_recovery_duration) - loan_amount_per_share) * total_warrants_bought
min_interest_on_loan = (loan_amount_per_share * (1 + daily_interst_rate) ** min(full_div_recovery_duration) - loan_amount_per_share) * total_warrants_bought

Date
1988-04-25    0.063600
1988-10-31    0.063600
1989-04-24    0.037038
1989-06-12    0.037038
1989-11-06    0.072018
                ...   
2020-09-03    0.754561
2021-03-04    1.311347
2021-09-02    2.715270
2022-02-24    2.080559
2022-09-01    2.551839
Name: Dividends, Length: 73, dtype: float64
[*********************100%***********************]  1 of 1 completed


### Analysis

In [7]:
analysis = f"In the past {len(ex_dates)} dividend payments, {successful_recovery} of the times \
{pyasx.data.companies.get_company_info(asx_code)['name'].strip('.')}'s share price managed to recover from its \
fall on the ex-dividend date before the next one, indicating a {round(success_rate * 100, 2)}% rate of successful recovery. \
On average (regardless whether the dividend was recvoered by the next ex-dividend date), \
the recovery often took {round(avg_recovery_duration)} days. The shortest \
recovery took {min(full_div_recovery_duration)} day(s), whilst the longest took {max(full_div_recovery_duration)} day(s)."

investment_analysis = f"According to the current {asx_code} share price, \
the first instalment per share is ${first_instalment}. When investing ${investment_amount} in CitiWarrant MINIs, \
you can buy {total_warrants_bought} warrants. The current annual interest rate on the loan is \
{round(annual_interest_rate_on_loan * 100, 2)}%, and the loan amount per share is ${loan_amount_per_share}. \
The upcoming dividend per share for {asx_code} is ${dps}, and {total_warrants_bought} \
warrants will result in ${round(total_div_received, 2)} \
of dividends. Based on the information above and the dividend recovery history of {asx_code}, \
the expected interest payable of this trade will be ${round(avg_interest_on_loan, 2)}. \
The best case will be ${round(min_interest_on_loan, 2)} of interest payable, whilst the worst case will be \
${round(max_interest_on_loan, 2)} of interest payable. The average profit will be ${round(total_div_received - avg_interest_on_loan, 2)}. \
The best portfit will be ${round(total_div_received - min_interest_on_loan, 2)}, whilst the worst will be ${round(total_div_received - max_interest_on_loan, 2)}."


print(f"Dividned Recovery Statistics of {asx_code} ({pyasx.data.companies.get_company_info(asx_code)['name'].strip('.')})")
print()
print(analysis)
print()
print("Summary")
print(f"Probability of successful dividend recovery before the next ex-date: {round(success_rate * 100, 2)}%")
print(f"Shortest recovery: {min(div_recovery_duration)} day(s)")
print(f"Longest recovery: {max(div_recovery_duration)} day(s)")
print()
print(investment_analysis)
print()
print("Summary")
print(f"Investment amount: ${investment_amount}")
print("Number of warrants bought:", total_warrants_bought)
print(f"Total dividend receivable (DPS ${dps}): ${round(total_div_received, 2)}")
print(f"Expected (average) interest payable: ${round(avg_interest_on_loan, 2)}")
print(f"Least interest payable: ${round(min_interest_on_loan, 2)}")
print(f"Most interest payable: ${round(max_interest_on_loan, 2)}")
print(f"Average profit: ${round(total_div_received - avg_interest_on_loan, 2)}")
print(f"Best profit: ${round(total_div_received - min_interest_on_loan, 2)}")
print(f"Worst profit: ${round(total_div_received - max_interest_on_loan, 2)}")

Dividned Recovery Statistics of BHP (BHP GROUP LIMITED)

In the past 73 dividend payments, 67 of the times BHP GROUP LIMITED's share price managed to recover from its fall on the ex-dividend date before the next one, indicating a 91.78% rate of successful recovery. On average (regardless whether the dividend was recvoered by the next ex-dividend date), the recovery often took 18 days. The shortest recovery took 1 day(s), whilst the longest took 1428 day(s).

Summary
Probability of successful dividend recovery before the next ex-date: 91.78%
Shortest recovery: 1 day(s)
Longest recovery: 163 day(s)

According to the current BHP share price, the first instalment per share is $25.94. When investing $10000 in CitiWarrant MINIs, you can buy 385 warrants. The current annual interest rate on the loan is 6.8%, and the loan amount per share is $20. The upcoming dividend per share for BHP is $2.5, and 385 warrants will result in $962.5 of dividends. Based on the information above and the dividend

### Graphs

Profit outcome analysis

In [8]:
# a line graph for the average return with different gearing ratios and profits
possible_loan_amounts = range(int(underlying_share_price * 0.1), int(underlying_share_price * 0.9)) # 10%-90% gearing
possible_first_instalments = []
possible_num_of_warrants = []
possible_average_profits = []
possible_best_profits = []
possible_worst_profits = []

successful_recovery_possible_average_profits = []
successful_recovery_possible_best_profits = []
successful_recovery_possible_worst_profits = []

for loan in possible_loan_amounts:
    possible_first_instalments.append(round(underlying_share_price - loan, 3))

for instalment in possible_first_instalments:
    possible_num_of_warrants.append(int(investment_amount / instalment))

for i in range(len(possible_num_of_warrants)):
    loan = possible_loan_amounts[i]
    div = dps * possible_num_of_warrants[i]
    
    avg_interest = (loan * (1 + daily_interst_rate) ** full_avg_recovery_duration - loan) * possible_num_of_warrants[i]
    best_interest = (loan * (1 + daily_interst_rate) ** min(full_div_recovery_duration) - loan) * possible_num_of_warrants[i]
    worst_interest = (loan * (1 + daily_interst_rate) ** max(full_div_recovery_duration) - loan) * possible_num_of_warrants[i]
    
    successful_recovery_avg_interest = (loan * (1 + daily_interst_rate) ** avg_recovery_duration - loan) * possible_num_of_warrants[i]
    successful_recovery_best_interest = (loan * (1 + daily_interst_rate) ** min(div_recovery_duration) - loan) * possible_num_of_warrants[i]
    successful_recovery_worst_interest = (loan * (1 + daily_interst_rate) ** max(div_recovery_duration) - loan) * possible_num_of_warrants[i]
    
    
    possible_average_profits.append(round(div - avg_interest, 2))
    possible_best_profits.append(round(div - best_interest, 2))
    possible_worst_profits.append(round(div - worst_interest, 2))
    
    successful_recovery_possible_average_profits.append(round(div - successful_recovery_avg_interest, 2))
    successful_recovery_possible_best_profits.append(round(div - successful_recovery_best_interest, 2))
    successful_recovery_possible_worst_profits.append(round(div - successful_recovery_worst_interest, 2))

result_df = pd.DataFrame()
result_df["Second Instalments $"] = possible_loan_amounts
result_df["Average Profit $"] = possible_average_profits
result_df["Best Profit $"] = possible_best_profits
result_df["Worst Profit $"] = possible_worst_profits

success_result_df = pd.DataFrame()
success_result_df["Second Instalments $"] = possible_loan_amounts
success_result_df["Average Profit After Successful Recovery $"] = successful_recovery_possible_average_profits
success_result_df["Best Profit After Successful Recovery $"] = successful_recovery_possible_best_profits
success_result_df["Worst Profit After Successful Recovery $"] = successful_recovery_possible_worst_profits

print("Profit outcomes at different loan amounts based on the entire dividend history")
px.line(result_df, x = "Second Instalments $", 
        y = ["Average Profit $", "Best Profit $", "Worst Profit $"],
        title = "Profit outcomes at different loan amounts").show()

print(f"Profit outcomes at {round(success_rate * 100, 2)}% of the chance (share price recovers before the next ex-dividend date)")
px.line(success_result_df, x = "Second Instalments $", 
        y = ["Average Profit After Successful Recovery $", "Best Profit After Successful Recovery $", "Worst Profit After Successful Recovery $"],
        title = f"Profit outcomes at different loan amounts ({round(success_rate * 100, 2)}% chance)").show()

Profit outcomes at different loan amounts based on the entire dividend history


Profit outcomes at 91.78% of the chance (share price recovers before the next ex-dividend date)


Break-even analysis

In [9]:
possible_recovery_duration = range(min(full_div_recovery_duration), max(full_div_recovery_duration))
profits = []

for duration in possible_recovery_duration:
    interest = (loan_amount_per_share * (1 + daily_interst_rate) ** duration - loan_amount_per_share) * total_warrants_bought
    profits.append(total_div_received - interest)
    
break_even_df = pd.DataFrame()
break_even_df["Recovery Duration (Days)"] = possible_recovery_duration
break_even_df["Profit $"] = profits

px.area(break_even_df, x = "Recovery Duration (Days)", y = "Profit $").show()